# PCA of white dwarf spectra

We will use the library of white dwarf spectra from Pierre Bergeron. The origin is http://www.astro.umontreal.ca/~bergeron/CoolingModels/.  The first functions here read the spectra from this sample in.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd
import seaborn as sns
import glob
from astropy.io import fits

%matplotlib inline

In [3]:
##
## The two first routines here are here in order to document
## the full process but you should not need to use them
##
def load_one_wd_spectrum(logg, T, silent=False):
    """Load one WD spectrum
    
    logg: The log surface gravity of the white dwarf
    T: The temperature to read in.
    silent: If set to True, a failure to find a file will just return in
            a return value of None, no message is printed out.
            
    Return: A dictionary with two keys: Wavelength and Flux. The wavelength is in Angstrom.
    
    """
    ROOT = './WhiteDwarfs/'
    
    pattern = ROOT+"bergeron_He_*_{0:2d}.dat_{1:d}".format(int(10*logg), int(T))
    
    files = glob.glob(pattern)
    try:
        fname = files[0]
    except:
        if not silent:
            print("I did not find anything corresponding to logg={0:.1f}, T={1}".format(logg, T))
        fname = None
    
    if fname is None:
        t = None
    else:
        tall = Table().read(fname, format='ascii', header_start=1,
                         names=('Wavelength', 'Flux'))
        tall['Wavelength'] = tall['Wavelength']*10 # Convert to AA
        # I only extract the UV-optical region.
        t = tall[(tall['Wavelength'] > 900) & (tall['Wavelength'] < 1e4) ]
        
    return t
    
def assemble_library():
    """
    Assemble the library of White dwarf spectra.
    
    Returns a tuple with the wavelength in Ångström, the flux, the logg and T.
    """
    # Extracted from file names with:
    # ls bergeron_He_* | perl -ane '{($logg, $T) = ($_ =~ m/bergeron_He_[0-9]+_([0-9]+)\.dat_([0-9]+)/); print $logg, "\n";}' | sort | uniq
    # and similar for the temperature
    loggs = [7.0, 7.5, 8.0, 8.5, 9.0]
    
    Ts = Table().read('./WhiteDwarfs/uniq.Ts', names=['T'], format='ascii',
                      data_start=0)
    
    n_logg = len(loggs)
    n_T = len(Ts)
    n_tot = n_T*n_logg
    counter = 0
    first = True
    logg_vec = np.zeros(n_tot)
    T_vec = np.zeros(n_tot)
    for i_g, logg in enumerate(loggs):
        for i_T, row in enumerate(Ts):
            T = row['T']
            sp = load_one_wd_spectrum(logg, float(T), silent=True)
            if first:
                spec = np.zeros((len(sp['Wavelength']), n_tot))
                first = False
                wave = sp['Wavelength']
                
            if sp is None:
                # Some combinations of temperature and logg do not exist
                n_tot = n_tot-1 
                continue
                
            logg_vec[counter] = logg
            T_vec[counter] = T
            spec[:, counter] = sp['Flux']
            counter = counter+1
            
    return wave, spec[:, 0:n_tot].T, logg_vec[0:n_tot], T_vec[0:n_tot]

def save_fits_compilation(l, f, logg, T):
    """For convenience, save a single FITS file with all the information
    """

    hdul = fits.HDUList()
    # Add a primary HDU
    hdul.append(fits.PrimaryHDU())

    names = ['Wave', 'Flux', 'Logg', 'Temp']
    for i, x in enumerate([l, f, logg, T]):
        hdu = fits.ImageHDU(x)
        hdu.name = names[i]
        hdul.append(hdu)

    hdul.writeto("WhiteDwarf_data.fits", overwrite=True)
    
    
def load_wd_spectra_from_fits():
    """Load in the FITS file created by save_fits_compilation
    
    Returns:
        l : numpy array
            The wavelength array (in Angstrom) with n_lambda elements
        f : numpy array
            The flux array with shape n_models x n_lambda
        logg : numpy array
            The surface gravity of each model with n_models elements.
        T : numpy array
            The effective temperature of each model with n_models elements
    """

    fname = "WhiteDwarf_data.fits"
    hdul = fits.open(fname)
    l = hdul['Wave'].data
    f = hdul['Flux'].data
    logg = hdul['Logg'].data
    T = hdul['Temp'].data
    return l, f, logg, T

In [4]:
# If you want to create the FITS file or load from the ASCII files
# directly you can uncomment the lines below.
# l, f, logg, T = assemble_library()
# save_fits_compilation(l, f, logg, T)

In [5]:
l, f, logg, T = load_wd_spectra_from_fits()

### a) Visualisation

Create a nice visualisation of the spectral library following the guidelines in the lectures. You may or may not show the whole library and there should be a way to infer the temperature of each model shown.

### b) PCA or NFM factorisation

Develop an algorithm to reconstruct the spectrum of the white dwarf in the optical region such that less than 5% of the spectra have bad reconstructions, where we define a bad reconstruction to be one where more than 5% of the wavelength points (in the optical) have  
$$\frac{\left|f_{\mathrm{true}}-f_{\mathrm{recon}}\right|}{f_{\mathrm{true}}} > 0.1$$

### c) Estimate PCA or NFM components using density estimators.

Create a kernel density estimate for each principal component and draw random principal components from this KDE. Implement this and explain why/why not this is a good way to create random spectra. Suggest one additional way to create random spectra and contrast the methods.
